In [10]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [2]:
train_data = pd.read_csv("train.csv")

In [3]:
train_data

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,55,165,60,81.0,0.5,0.6,1,1,135,...,40,75,16.5,1,1.0,22,25,27,0,1
1,1,70,165,65,89.0,0.6,0.7,2,2,146,...,57,126,16.2,1,1.1,27,23,37,1,0
2,2,20,170,75,81.0,0.4,0.5,1,1,118,...,45,93,17.4,1,0.8,27,31,53,0,1
3,3,35,180,95,105.0,1.5,1.2,1,1,131,...,38,102,15.9,1,1.0,20,27,30,1,0
4,4,30,165,60,80.5,1.5,1.0,1,1,121,...,44,93,15.4,1,0.8,19,13,17,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159251,159251,40,155,45,69.0,1.5,2.0,1,1,127,...,72,159,14.5,1,0.8,25,26,13,0,0
159252,159252,50,155,75,82.0,1.0,1.0,1,1,120,...,64,108,14.5,1,0.6,21,20,18,0,0
159253,159253,40,160,50,66.0,1.5,1.0,1,1,114,...,87,93,10.9,1,0.6,15,9,12,0,0
159254,159254,50,165,75,92.0,1.2,1.0,1,1,121,...,55,80,14.4,1,1.1,22,17,37,0,1


In [13]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tqdm import tqdm
import torch



class TrainDataset(Dataset):
    def __init__(self, data):
        self.labels = torch.tensor(data.loc[:, "smoking"].values, dtype=torch.float32)
        self.id = torch.tensor(data["id"].values)
        self.data = torch.tensor(data.drop(["smoking", "id"], axis=1).values, dtype=torch.float32)
    
    def __getitem__(self, idx):
        return {"data": self.data[idx], "label": self.labels[idx], "id": self.id[idx]}
    
    def __len__(self):
        return len(self.data)
    
train_dataset = TrainDataset(train_data)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2000)

print(torch.cuda.is_available())

False


In [14]:
class TestDataset(Dataset):
    def __init__(self, data):
        self.id = torch.tensor(data["id"].values)
        self.data = torch.tensor(data.drop(["id"], axis=1).values, dtype=torch.float32)
    
    def __getitem__(self, idx):
        return {"data": self.data[idx], "id": self.id[idx]}
    
    def __len__(self):
        return len(self.data)

In [15]:
test_data = pd.read_csv("test.csv")
test_dataset = TestDataset(test_data)
test_dataloader = DataLoader(test_dataset, batch_size=1000)

In [16]:
class MLP(nn.Module):
    
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(num_features, 500),
            nn.Dropout(0.5),
            nn.Linear(500, 500),
            nn.ReLU(),
            nn.Linear(500, 1),
        )
        
    def forward(self, x):
        return self.mlp(x)

In [120]:
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR

device = "cuda" if torch.cuda.is_available() else "cpu"
model = MLP(22, 1)
optimizer = Adam(model.parameters(), lr=1e-2)
loss_fn = nn.BCEWithLogitsLoss()
lr_scheduler = ExponentialLR(optimizer, gamma=0.9)
print(device)

cpu


In [116]:
model.train()
for i in range(15):
    training_loss = 0
    for batch in tqdm(train_dataloader):   
        x, y = batch["data"].to(device), batch["label"].unsqueeze(1).to(device)
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
    
    lr_scheduler.step()

    print("Training loss after epoch {} equals {}".format(i, training_loss / len(train_dataloader)))
    

100%|██████████| 80/80 [00:11<00:00,  6.69it/s]


Training loss after epoch 0 equals 0.5617524839937686


100%|██████████| 80/80 [00:11<00:00,  6.69it/s]


Training loss after epoch 1 equals 0.5392203465104103


100%|██████████| 80/80 [00:11<00:00,  6.74it/s]


...

Training loss after epoch 12 equals 0.5159890331327915


100%|██████████| 80/80 [00:11<00:00,  6.71it/s]


Training loss after epoch 13 equals 0.515370125323534


100%|██████████| 80/80 [00:11<00:00,  6.71it/s]

Training loss after epoch 14 equals 0.5152473080903291


In [117]:
with open("out.csv", "w") as f:
    f.write("id,smoking\n")

model.eval()

MLP(
  (mlp): Sequential(
    (0): Linear(in_features=22, out_features=500, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): Tanh()
    (3): Linear(in_features=500, out_features=500, bias=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): Tanh()
    (6): Linear(in_features=500, out_features=500, bias=True)
    (7): Linear(in_features=500, out_features=1, bias=True)
  )
)

In [118]:
with open("out.csv", "a") as f:
    with torch.no_grad():
        for batch in test_dataloader:
            id_, data = batch["id"], batch["data"]
            pred = nn.Sigmoid()(model(data)).tolist()
            for i, p in zip(id_, pred):
                f.write(str(i.item()) + ',' + str(round(p[0], 1)) + '\n')